In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class nystrom:
    def __init__(self, n, n_int, a, b):
        #Initialize n+1 x n+1 matrix with all zeros
        self.matrix = np.zeros((n+1,n+1))
        self.t_values = np.zeros(n+1)
        self.weights = np.zeros(n+1)
        self.y_values = np.zeros(n+1)
        self.solution = np.zeros(n+1)
        self.true_solution = np.zeros(n+1)
        self.h = (b-a)/n
        self.int_x_values = np.zeros(n_int+1)
        
    def set_t_values(self):
        for i in range(0,n+1):
            self.t_values[i] = a + i*self.h

    def set_weights(self):
        self.weights[0] = 0.5
        self.weights[n] = 0.5
        for i in range(1,n):
            self.weights[i] = 1
        self.weights = self.weights*self.h;
    
    def build_matrix(self):
        for i in range(0,n+1):
            for j in range(0,n+1):
                self.matrix[i,j] = self.weights[j]*np.exp(self.t_values[i]*self.t_values[j])
        
        self.matrix = -self.matrix + 50*np.identity(n+1)
        
    def set_y_values(self):
        for i in range(0,n+1):
            self.y_values[i] = 50*np.exp(self.t_values[i]) - (np.exp(self.t_values[i]+1))/(self.t_values[i]+1) + 1/(self.t_values[i]+1)
    
    def y_function(self,t):
        return 50*np.exp(t) - (np.exp(t+1))/(t+1) + 1/(t+1)
    
    def solve_linear_system(self):
        matrix_inverse = np.linalg.inv(self.matrix)
        self.solution = np.matmul(matrix_inverse,self.y_values)
        return self.solution
    
    def nys_interp(self):
        int_t_values = np.zeros(n_int+1)
        h_int = (b-a)/n_int
        for i in range(0,n_int+1):
            int_t_values[i] = a + i*h_int
            asum = 0
            for j in range(0,n+1):
                asum = asum + (1/50)*self.weights[j]*np.exp(int_t_values[i]*self.t_values[j])*self.solution[j]
            
            asum = asum + self.y_function(int_t_values[i])/50
            self.int_x_values[i] = asum
        
        return self.int_x_values
    
    def interp_error(self):
        h_int = (b-a)/n_int
        true_sol_int = np.zeros(n_int+1)
        int_t_values = np.zeros(n_int+1)
        
        for i in range(0,n_int+1):
            int_t_values[i] = a + i*h_int
            true_sol_int[i] = np.exp(int_t_values[i])
        
        error = np.zeros(n_int+1)
        for i in range(0,n_int+1):
            error[i] = np.abs(true_sol_int[i] - self.int_x_values[i])    
        
        #return np.amax(error)
        return error
    
    def compute_error(self):
        for i in range(0,n+1):
            self.true_solution[i] = np.exp(self.t_values[i])
            
        error = np.zeros(n+1)
        for i in range(0,n+1):
            error[i] = np.abs(self.true_solution[i] - self.solution[i])
            
        return np.amax(error)
    
    def return_t(self):
        return self.t_values
        


In [ ]:
n = 50
n_int = 10000
a = 0
b = 1
nys_test = nystrom(n,n_int,a,b)
nys_test.set_t_values()
nys_test.set_weights()
nys_test.build_matrix()
nys_test.set_y_values()

x = nys_test.solve_linear_system()
xn = nys_test.nys_interp()
t = nys_test.return_t()
error = nys_test.compute_error()
error_int = nys_test.interp_error()

#print(x)
#print(xn)
#print(t)
#print(error)
print(error_int)

In [ ]:
pq1=plt.plot(t,x,'ro',markersize=5,label='Numerical solution')
pq2=plt.plot(np.linspace(0,1,10001),np.exp(np.linspace(0,1,10001)),'k-',label='True solution')
#plt.title('Interpolated and Exact solution vs. t')
plt.legend()
plt.xlabel('t')
plt.ylabel('x(t)')

In [ ]:
#plt.plot(t,x)

p1=plt.plot(np.linspace(0,1,10001),xn,'ro',markersize=5,label='Interpolated values')
p2=plt.plot(np.linspace(0,1,10001),np.exp(np.linspace(0,1,10001)),'k-',label='True solution')
#plt.title('Interpolated and Exact solution vs. t')
plt.legend()
plt.xlabel('t')
plt.ylabel('x(t)')

In [ ]:
p3 = plt.plot(np.linspace(0,1,10001),error_int,'b-',markersize=5,label='Interpolation Error')
plt.xlabel('t')
plt.ylabel('Interpolation error')
plt.yscale('log')

In [ ]:
e1 = np.zeros(4)
e1[0] = 0.005350901772309236
e1[1] = 0.0013513738476640391
e1[2] = 0.0003387120189350945
e1[3] = 8.473253532104152e-05

e2 = np.zeros(4)
e2[0] = 0.0007662692264425175
e2[1] = 0.0001917725431808126
e2[2] = 4.799633051355556e-05
e2[3] = 1.202302039038372e-05

nvals = np.zeros(4)
nvals[0] = 2
nvals[1] = 4
nvals[2] = 8
nvals[3] = 16

p5=plt.plot(nvals,e1,'bo',markersize=5,label='E1')
p6=plt.plot(nvals,e2,'ro',markersize=5,label='E2')
nvecfloat=np.array([2.0,4.0,8.0,16.0,32.0,64.0])
p7=plt.plot(nvecfloat,np.power(nvecfloat,(-2)),'g-',label='Line with slope 2')
plt.legend()
plt.xlabel('n')
plt.ylabel('error')
plt.yscale('log')
plt.xscale('log')

In [5]:
import numpy as np
import sympy as sp

In [11]:
class nystrom_general:
    def __init__(self, n, a, b, lam, x1, x2):
        self.lam = lam
        self.n = n
        self.a = a
        self.b = b
        self.h = (b-a) / n
        self.Matrix = np.zeros((n+1, n+1))
        self.t = sp.Symbol('t')
        self.s = sp.Symbol('s')
        self.x1 = x1(self.s)
        self.x2 = x2(self.s)
        self.y1 = x1(self.t)
        self.y2 = x2(self.t)
        self.y = np.zeros(n+1)
        self.normal = [-1*sp.diff(self.y2,self.t), sp.diff(self.y1,self.t)]
        self.greenfxn = (1/(2*np.pi))*sp.log(sp.sqrt(sp.Pow(self.x1-self.x2,2)+sp.Pow(self.y1-self.y2,2)))
        

In [10]:
nys = nystrom_general(8, 0, 1, -0.5, sp.cos, sp.sin)
print(nys.greenfxn)

0.159154943091895*log(sqrt((-sin(s) + cos(s))**2 + (-sin(t) + cos(t))**2))
